In [1]:
import numpy as np
import pandas as pd  
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/Glass_Quality_Participants_Data/Test.csv
/kaggle/input/Glass_Quality_Participants_Data/Sample_Submission.xlsx
/kaggle/input/Glass_Quality_Participants_Data/Train.csv


In [2]:
train = pd.read_csv('/kaggle/input/Glass_Quality_Participants_Data/Train.csv')
test = pd.read_csv('/kaggle/input/Glass_Quality_Participants_Data/Test.csv')
sub = pd.read_excel('/kaggle/input/Glass_Quality_Participants_Data/Sample_Submission.xlsx')

In [3]:
train.shape, test.shape, sub.shape

((1358, 16), (583, 15), (583, 2))

In [4]:
train.head(3)

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,class
0,0,1,150,36,144,172,947225,947332,439,439.099273,0,0,1,0,0,1
1,1,0,134,55,1144,1152,2379058,2379624,329,329.205616,1,0,0,0,0,1
2,1,0,135,65,950,974,1038442,1036754,300,300.120598,0,0,0,0,0,2


In [5]:
train['class'].value_counts()

1    887
2    471
Name: class, dtype: int64

In [6]:
X = train.drop(labels=['class'], axis=1)
y = train['class'].values

In [7]:
Xtest = test

In [8]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.metrics import log_loss

In [9]:
errlgb = []
y_pred_totlgb = []

fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in fold.split(X, y):
    
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = ExtraTreesClassifier(random_state=42) 
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)
    print("Log Loss: ", (log_loss(y_test, y_pred)))
    
    errlgb.append(log_loss(y_test, y_pred))
    p = clf.predict_proba(Xtest)
    y_pred_totlgb.append(p)

Log Loss:  0.3019055153132171
Log Loss:  0.29887578610536286
Log Loss:  0.2806766272415199
Log Loss:  0.21948721616275454
Log Loss:  0.3388815552294614
Log Loss:  0.2503933542465036
Log Loss:  0.2492569282368466
Log Loss:  0.3187786303961658
Log Loss:  0.291456237489663
Log Loss:  0.29780674038124083


In [10]:
np.mean(errlgb,0)

0.2847518590802735

In [11]:
y_pred = np.mean(y_pred_totlgb,0)

In [12]:
pd.DataFrame(y_pred).rename(columns={0:1, 1:2}).to_excel('Output.xlsx', index=False)